In [1]:
pip install torch_geometric

  Obtaining dependency information for torch_geometric from https://files.pythonhosted.org/packages/65/4e/6f9a75548a93fedcd4514ae2de9bee1e91bade6b73252b4da32f0e42ac52/torch_geometric-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch_geometric 
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.utils import to_dense_batch

from sklearn.preprocessing import LabelEncoder


df = pd.read_csv('/kaggle/input/inn-competition/Trainset-Evaluation/Train_Set.csv')
ts = pd.read_csv('/kaggle/input/inn-competition/Transaction/InnoTech_Trans.csv')
# For prototype phase we remove the nan values
ts=ts.dropna()
ts = ts.reset_index().drop(columns='index')

# TEMP LINE CODE
df = df.drop_duplicates(subset=['CARD'])

ts = ts.drop(columns=['Terminal_ID'])

In [3]:
le = LabelEncoder()  
le.fit(np.unique(np.concatenate([ts['Primary_ID'].values,ts['Second_ID'].values,df['CARD'].values])).tolist())

ts['Primary_ID'] = le.transform(ts['Primary_ID'])
ts['Second_ID'] = le.transform(ts['Second_ID'])
df['CARD'] = le.transform(df['CARD'])

# ts = pd.get_dummies(ts, columns=['Date'], prefix='Date')

In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

encoder = OneHotEncoder(sparse=False, drop='first')  # 'first' to drop one of the columns to avoid multicollinearity

column_transformer = ColumnTransformer(
    transformers=[('onehot', encoder, ['Date'])],
    remainder='passthrough'  # keep the other columns as they are
)

df_encoded = pd.DataFrame(column_transformer.fit_transform(ts))


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
ts=pd.concat([ts,df_encoded],axis=1).drop(columns=['Date'])

In [6]:
df['CARD']

0          2886
3         15128
4         51645
5         59418
6         59746
          ...  
28178    125944
28181    468893
28182    555196
28183    922029
28184    522065
Name: CARD, Length: 3235, dtype: int64

In [20]:
card_to_index = ts[ts['Primary_ID']==51645].index
card_in_index = ts[ts['Second_ID']==51645].index

edge_index_to,edge_feature_to = ts.loc[card_to_index,['Primary_ID','Second_ID']].values,ts.loc[card_to_index].values
edge_index_in,edge_feature_in = ts.loc[card_in_index,['Primary_ID','Second_ID']].values,ts.loc[card_in_index].values

edge_indexs = np.concatenate([edge_index_in,edge_index_to])

nodes = np.concatenate([np.unique((ts.loc[card_to_index,['Second_ID']])),
                        np.unique((ts.loc[card_in_index,['Primary_ID']])),np.array([51645])])


(222,)

In [148]:
nodes = np.concatenate([np.unique((ts.loc[card_to_index,['Second_ID']])),
                        np.unique((ts.loc[card_in_index,['Primary_ID']])),np.array([51645])])

node_dict = {node: index for index, node in enumerate(nodes)}

formal_edge_index = [[node_dict[src], node_dict[dst]] for src, dst in edge_indexs]


nodes = torch.tensor(nodes)
edge_indexs = torch.tensor(formal_edge_index, dtype=torch.long).t().contiguous()


KeyError: tensor(216)

In [149]:
edge_indexs = torch.tensor(formal_edge_index, dtype=torch.long).t().contiguous()


In [177]:
# edge_indexs.reshape(-1,2)

In [7]:
import torch
from torch_geometric.data import Data

def construct_graph(card_id,y):
    """
    This function will create a graph of transaction for each card base on the transaction dataset. The created graph
    is directed which means each CARD_ID can be source or destination of the transaction.

    params:
        card_id 
        ts: transaction dataset

    return:

        graph 
    """

    card_to_index = ts[ts['Primary_ID']==card_id].index
    card_in_index = ts[ts['Second_ID']==card_id].index

    edge_index_to,edge_feature_to = ts.loc[card_to_index,['Primary_ID','Second_ID']].values,ts.loc[card_to_index].values
    edge_index_in,edge_feature_in = ts.loc[card_in_index,['Primary_ID','Second_ID']].values,ts.loc[card_in_index].values

    edge_indexs = np.concatenate([edge_index_in,edge_index_to])

    nodes = np.concatenate([np.unique((ts.loc[card_to_index,['Second_ID']])),
                            np.unique((ts.loc[card_in_index,['Primary_ID']])),np.array([card_id])])
    
    node_dict = {node: index for index, node in enumerate(nodes)}
    
    formal_edge_index = [[node_dict[src], node_dict[dst]] for src, dst in edge_indexs]
    

    nodes = torch.tensor(nodes.re)
    # convert (n,2) to (2,n) shape 
    edge_indexs = torch.tensor(formal_edge_index, dtype=torch.long).reshape(-1,2).t().contiguous()
    
    edge_features = torch.tensor(np.concatenate([edge_feature_to,edge_feature_in]))
    
    data = Data(x=nodes, edge_index=edge_indexs,y=y)
    data.edge_attr = edge_features

    return data


In [190]:
data[0]

Data(x=[880], edge_index=[2, 1068], y=1.0, edge_attr=[1068, 104])

In [8]:
from tqdm import tqdm

data = []

for (i, y) in tqdm(zip(df['CARD'], df['LABEL']), total=len(df)):
    data.append(construct_graph(i, y))


100%|██████████| 3235/3235 [03:50<00:00, 14.02it/s]


In [9]:
# Save the prepared data
torch.save(data,'/kaggle/working/graph_data.pt')



In [201]:
import os
import torch
from torch_geometric.data import Dataset, Data
from torch_geometric.utils import to_dense_batch

class CustomGraphDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.graph_files =root
        self.data = torch.load(self.graph_files)
        self.max_num_nodes = self.find_max_num_nodes()
        self.max_num_edge_index = self.find_max_num_edge_index()
        
    def find_max_num_edge_index(self):
        max_num_edge_index = max(data.edge_index.shape[1] for data in self.data)
        return max_num_edge_index
    
    def find_max_num_nodes(self):
        max_num_nodes = max(data.num_nodes for data in self.data)
        return max_num_nodes
    
    def indices(self):
        return list(range(self.len()))

    def len(self):
        return len(self.data)

    def get(self, idx):
        
        num_nodes_to_pad = self.max_num_nodes - self.data[idx].num_nodes
        num_edge_to_pad = self.max_num_edge_index - self.data[idx].edge_index.shape[1]
        
        # Pad num_nodes with zeros
        self.data[idx].x = torch.cat([self.data[idx].x, torch.zeros(num_nodes_to_pad)])
        
        # Pad edge_index with zeros
        self.data[idx].edge_index = torch.cat([self.data[idx].edge_index, torch.zeros(2,num_edge_to_pad)],dim=1)
        self.data[idx].edge_attr = torch.cat([self.data[idx].edge_attr,torch.zeros(num_edge_to_pad,104)])
        # Update the number of nodes in the data object
        self.data[idx].num_nodes = self.max_num_nodes

        return self.data[idx]

In [202]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_dense_batch

def collate_fn(batch):
    batch = to_dense_batch(batch)
    return Data(x=batch.x, edge_index=batch.edge_index, y=batch.y)

In [203]:
root = '/kaggle/working/graph_data.pt'
train_dataset = CustomGraphDataset(root)
train_dataset.max_num_nodes

24651

In [204]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# for step, data in enumerate(train_loader):
#     print(f'Step {step + 1}:')
#     print('=======')
#     print(f'Number of graphs in the current batch: {data.num_graphs}')
#     print(data)
#     print()

In [10]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(data, batch_size=64, shuffle=True)

In [ ]:
for 

In [62]:
def weights_init(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)


In [90]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool

class GAT(torch.nn.Module):
    def __init__(self, node_input_channels, edge_input_channels, hidden_channels, num_heads):
        super(GAT, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(node_input_channels, hidden_channels, heads=num_heads, edge_input_channels=edge_input_channels)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, edge_input_channels=edge_input_channels)
        self.conv3 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, edge_input_channels=edge_input_channels)
        self.lin = Linear(hidden_channels * num_heads, 3)

    def forward(self, x, edge_index, edge_attr, batch):
        x = self.conv1(x, edge_index, edge_attr)
        x = F.elu(x)  # Use ELU as activation for GAT
        x = self.conv2(x, edge_index, edge_attr)
        x = F.elu(x)
        x = self.conv3(x, edge_index, edge_attr)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x


model = GAT(1,104,64,2)

model.apply(weights_init)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
#          print(data.x.shape)
#          print(data.edge_index.shape)
#          print(data.edge_attr.shape)
#          print(data.x.shape)

         out = model(data.x.reshape(data.x.shape[0],1).float(), data.edge_index,data.edge_attr, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y.long())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x.reshape(data.x.shape[0],1).float(), data.edge_index, data.batch)  # Perform a single forward pass.
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y.long()).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


GAT(
  (conv1): GATConv(1, 64, heads=2)
  (conv2): GATConv(128, 64, heads=2)
  (conv3): GATConv(128, 64, heads=2)
  (lin): Linear(in_features=128, out_features=3, bias=True)
)


In [ ]:
for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
#     test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')

In [80]:
22137*104

2302248